# Chord voicings

I will speak of chords in a way that is much less nuanced than how they're typically spoken of in music theory.

For our purposes, there are two ideas that fall under the category of "chords".

The first are what can be thought of as _classes_ of chords, which are represented by pitch class sets, and are essentially equivalent to what I typically call "scales".

The second are what can be thought of as chord _voicings_, which are represented by pitch sets. 

You can have a chord like Cmaj7, represented by the pitch class set `{0, 4, 7, 11} mod 12 root 0`.

Think of this as an entire class of chords. From it, we can build many specific voicings by constructing pitch sets from this pitch class set.

## A notation for writing out voicings

I've come to find that it's easiest to write out voicings by permuting the pitch classes, and then translating them to pitches.

For example: `[0, 4, 11, 7]` is a permutation of pitch classes (mod 12) which represents the pitch set `{0, 4, 11, 19}`.

This simplifies the algorithm for enumerating chord voicings, because it simply becomes a matter of enumerating permutations of pitch classes.

In order to translate these into pitch sets, we'll need a function that adds an appropriate number of octaves to each pitch class.

In [1]:
from harmonica.pitch import PitchSet


def pitch_class_permutation_to_pitch_set(pc_perm: list[int], modulus: int) -> PitchSet:
    assert all(
        [0 <= pitch_class < modulus for pitch_class in pc_perm]
    ), "Pitch classes must be non-negative and less than modulus."
    assert modulus > 0, "Modulus must be positive."

    pitches = [pc_perm[0]]

    octave = 0

    for i, pitch_class in enumerate(pc_perm[1:]):
        if pitch_class <= pc_perm[i]:
            octave += 1
        pitches.append(pitch_class + (modulus * octave))

    return PitchSet(pitches)


voicing = pitch_class_permutation_to_pitch_set([4, 11, 7, 4, 2], 12)
print(voicing)  # {4, 11, 19, 28, 38}

PitchSet(pitches=[4, 11, 19, 28, 38])


## Enumerating all voicings of a pitch class set with no repetitions

In [2]:
from itertools import permutations
from harmonica.pitch import PitchClassSet, PitchSetSeq
from harmonica.utility import GM


def enumerate_voicings_no_repetitions(pitch_class_set: PitchClassSet) -> list[PitchSet]:
    pitch_class_permutations = [
        list(perm) for perm in permutations(pitch_class_set.pitch_classes)
    ]

    pitch_sets = [
        pitch_class_permutation_to_pitch_set(pc_perm, pitch_class_set.modulus)
        for pc_perm in pitch_class_permutations
    ]

    return pitch_sets


pitch_class_set = PitchClassSet([0, 4, 7, 11], modulus=12)
voicings = enumerate_voicings_no_repetitions(pitch_class_set)

for voicing in voicings:
    print(voicing)

# PitchSetSeq(voicings).preview(bass=48, program=GM.ElectricPiano1)

PitchSet(pitches=[0, 4, 7, 11])
PitchSet(pitches=[0, 4, 11, 19])
PitchSet(pitches=[0, 7, 16, 23])
PitchSet(pitches=[0, 7, 11, 16])
PitchSet(pitches=[0, 11, 16, 19])
PitchSet(pitches=[0, 11, 19, 28])
PitchSet(pitches=[4, 12, 19, 23])
PitchSet(pitches=[4, 12, 23, 31])
PitchSet(pitches=[4, 7, 12, 23])
PitchSet(pitches=[4, 7, 11, 12])
PitchSet(pitches=[4, 11, 12, 19])
PitchSet(pitches=[4, 11, 19, 24])
PitchSet(pitches=[7, 12, 16, 23])
PitchSet(pitches=[7, 12, 23, 28])
PitchSet(pitches=[7, 16, 24, 35])
PitchSet(pitches=[7, 16, 23, 24])
PitchSet(pitches=[7, 11, 12, 16])
PitchSet(pitches=[7, 11, 16, 24])
PitchSet(pitches=[11, 12, 16, 19])
PitchSet(pitches=[11, 12, 19, 28])
PitchSet(pitches=[11, 16, 24, 31])
PitchSet(pitches=[11, 16, 19, 24])
PitchSet(pitches=[11, 19, 24, 28])
PitchSet(pitches=[11, 19, 28, 36])


## Including voicings with repeated pitch classes

Obviously, this enumeration doesn't account for all possible ways to voice a pitch class set. There are many voicings in which a pitch class occurs multiple times. 

On a purely mathematical level (which is the level I enjoy working on), there are infinitely many occurrences of a pitch class in the pitch domain. 
Therefore, in order to have a finite enumeration, we must specify some maximum number of repetitions as a parameter.

### Scratch work

in the previous algo, I just take permutations of the unique set of pitch classes. 

im not really seeing an existing function in itertools that does what I want, sooo...

here's an example of what I want. I want to input a pitch class set, and a max_repetitions parameter that represents
now many times each pitch class can occur in the permutation.

for example, if the pcset is {0,4,7} and the max repetitions is 2, then the results are:

```
0 4 7
0 7 4
4 0 7
4 7 0
7 0 4
7 4 0
0 4 7 7
0 7 4 7
0 7 7 4
4 0 7 7
...
```

ah. its occurred to me that I can break this down and make it simpler.

so, really, this is a matter of multiset permutations.

and what I'm doin is generating all the multisets from the original set, {0,4,7}, where the maximum occurrences of each element is 2.

I'm enumerating all of the permutations of these multisets, and then returning the union of all these permutations.

So, the first step is generating all the multisets. you start by taking the first set, which has all distinct elements. {0,4,7}

then you can begin building the multisets.

I think what you do here is, first, you check the repetitions value. if it's equal to 1, then you just call the previous algorithm. 

if not, then you repeat the following process for each extra repetition:

take the powerset of the set. then, extend each of these combinations with the original set.

in our example, the powerset is 

`[(), (0,), (4,), (7,), (0, 4), (0, 7), (4, 7), (0, 4, 7)]`

so we extend each of these with the original set to get 

```
0 4 7
0 4 7 0
0 4 7 4
0 4 7 7
0 4 7 0 4
0 4 7 0 7
0 4 7 4 7
0 4 7 0 4 7
```

if max_repetitions == 2, then we have our complete list of multisets.

however, for each successive value of max_repetitions, we must do the same thing again, but on each value in the list.

for this reason, I think it's actually wise to ignore the empty set. we'll take `powerset[1:]`.

man, this is tricky!

isnt there a simpler way to do this?

like, can't we represent the presence of elements as numbers?

so for the set {0,4,7}
and all multisets with occurrences 1 through max_reps, lets say 3 here,
itd be like, 1 1 1, 1 1 2, 1 1 3, 1 2 1, 1 2 2, 1 2 3, 1 3 1, 1 3 2, 1 3 3, 2 1 1, etc...
so like, the third cartesian power of {1,2,3}, right?

In [3]:
# from itertools import product


# def rep_power(l: int, r: int):
#     return product(range(1, r + 1), repeat=l)


# p = rep_power(3, 4)
# for i in p:
#     print(i)

okay, that works. now how do I take a list like (1,3,1) and a set like {0,4,7} and return {0,4,4,4,7}?

In [4]:
# def multisetter(s: list[int], reps: list[int]) -> list[int]:
#     result: list[int] = []

#     for i, item in enumerate(s):
#         for _ in range(reps[i]):
#             result.append(item)

#     return result


# print(multisetter([0, 4, 7], [7, 3, 1]))

okay, cool. so now I should be able to use each of these to generate the set of all multisets that I desire.

In [5]:
# def multisets_with_max_repetitions(
#     original_set: list, max_repetitions: int
# ) -> list[list]:
#     multisets: list[list] = []

#     rep_counts = product(range(1, max_repetitions + 1), repeat=len(original_set))

#     for rep_count in rep_counts:
#         multiset = []
#         for i, item in enumerate(original_set):
#             for _ in range(rep_count[i]):
#                 multiset.append(item)
#         multisets.append(multiset)

#     return multisets


# multisets = multisets_with_max_repetitions([0, 4, 7], 4)
# for multiset in multisets:
#     print(multiset)

That seems to work - we're getting closer!

Now we just need to get all the distinct permutations of these results.

In [6]:
# from more_itertools import distinct_permutations

# permutations = []
# multisets = multisets_with_max_repetitions([0, 4, 7], 2)
# for multiset in multisets:
#     permutations += list(distinct_permutations(multiset))

# for perm in permutations:
#     print(perm)

Alright. Let's put it all together:

In [ ]:
from fractions import Fraction
from harmonica.pitch import PitchClassSet, PitchSet
from more_itertools import distinct_permutations
from itertools import product


def multiset_permutations_with_max_repetitions(
    original_set: list, max_repetitions: int
) -> list[list]:
    assert max_repetitions > 0, "Maximum repetitions must be positive."

    if original_set == []:
        return []

    multisets = []

    rep_counts = product(range(1, max_repetitions + 1), repeat=len(original_set))

    for rep_count in rep_counts:
        multiset = []
        for i, item in enumerate(original_set):
            for _ in range(rep_count[i]):
                multiset.append(item)
        multisets.append(multiset)

    permutations = []

    for multiset in multisets:
        permutations += list(distinct_permutations(multiset))

    return permutations


def enumerate_voicings_with_repetitions(
    pitch_class_set: PitchClassSet, max_repetitions: int
) -> list[PitchSet]:
    assert max_repetitions > 0, "Maximum repetitions must be positive."

    pitch_class_permutations = [
        list(perm)
        for perm in multiset_permutations_with_max_repetitions(
            pitch_class_set.pitch_classes, max_repetitions
        )
    ]

    pitch_sets = [
        pitch_class_permutation_to_pitch_set(pc_perm, pitch_class_set.modulus)
        for pc_perm in sorted(pitch_class_permutations, key=lambda x: (len(x), sum(x)))
    ]

    return pitch_sets


pitch_class_set = PitchClassSet([0, 4, 7], modulus=12)
voicings = enumerate_voicings_with_repetitions(pitch_class_set, 3)

print(len(voicings))
for voicing in voicings:
    print(voicing)


PitchSetSeq(voicings).preview(bass=36, program=GM.Clarinet, duration=Fraction("1"))

5052
PitchSet(pitches=[0, 4, 7])
PitchSet(pitches=[0, 7, 16])
PitchSet(pitches=[4, 12, 19])
PitchSet(pitches=[4, 7, 12])
PitchSet(pitches=[7, 12, 16])
PitchSet(pitches=[7, 16, 24])
PitchSet(pitches=[0, 12, 16, 19])
PitchSet(pitches=[0, 12, 19, 28])
PitchSet(pitches=[0, 4, 12, 19])
PitchSet(pitches=[0, 4, 7, 12])
PitchSet(pitches=[0, 7, 12, 16])
PitchSet(pitches=[0, 7, 16, 24])
PitchSet(pitches=[4, 12, 24, 31])
PitchSet(pitches=[4, 12, 19, 24])
PitchSet(pitches=[4, 7, 12, 24])
PitchSet(pitches=[7, 12, 24, 28])
PitchSet(pitches=[7, 12, 16, 24])
PitchSet(pitches=[7, 16, 24, 36])
PitchSet(pitches=[0, 4, 16, 19])
PitchSet(pitches=[0, 4, 7, 16])
PitchSet(pitches=[0, 7, 16, 28])
PitchSet(pitches=[4, 12, 16, 19])
PitchSet(pitches=[4, 12, 19, 28])
PitchSet(pitches=[4, 16, 24, 31])
PitchSet(pitches=[4, 16, 19, 24])
PitchSet(pitches=[4, 7, 12, 16])
PitchSet(pitches=[4, 7, 16, 24])
PitchSet(pitches=[7, 12, 16, 28])
PitchSet(pitches=[7, 16, 24, 28])
PitchSet(pitches=[7, 16, 28, 36])
PitchSet(pitche